This file is used to create and pickle the 'base' graph files and shortest paths so they can be used later

In [ ]:
import pandas as pd
import networkx as nx
import scripts as scr
import fileloader as fl

In [ ]:
import importlib
importlib.reload(fl)
importlib.reload(scr)
import fileloader as fl
import scripts as scr
import matplotlib as plt

First, grabbing the required files using fileloader.py

In [ ]:
graph = fl.get_network()
od = fl.get_OD_LU()
frequency = fl.get_frequency()

Next, we want to save the 'basegraphs' - unaltered network graphs which have their capacity assigned. 

In [ ]:
basegraphs = scr.create_base_graphs(graph, frequency)

In certain (late) timeslots, due to very low train frequency, capacity is at 0. This skews results, therefore we iteratively take the average capacity of neighbouring links which sligtly increases average capacity, but allows for better analysis of results. 

In [ ]:
def fixCapacity(bgraph, depth):
    depth += 1 #keep track of depth to prevent stack overflow
    #create a list of edges with no capacity
    nocap = [(a, b) for a, b, data in bgraph.edges(data=True) 
        if data['capacity'] == 0] 

    for start, end in nocap:
        sneighbors = [n for n in bgraph.neighbors(start)] 
        eneighbors = [n for n in bgraph.neighbors(end)]
        #average of neighbours at start of link
        savg = sum(bgraph[s][start]['capacity'] 
            for s in sneighbors)/len(sneighbors) 

        #average of neighbours at end of link
        eavg = sum(bgraph[e][end]['capacity'] 
            for e in eneighbors)/len(sneighbors) 

        bgraph[start][end]['capacity'] = (savg + eavg)//2
    if depth > 100: #if it doesn't converge at 100 iterations, stop
        print('no solution found')
        return
    if len(nocap) > 0: 
        fixCapacity(bgraph, depth)
    return

for bgraph in basegraphs.values(): 
    fixCapacity(bgraph, 0)

In [ ]:
fl.save_obj(basegraphs, 'basegraphs')

One of the more 'heavier' calculations is getting the shortest path for the ~68k OD-pairs. Therefore, we also want to save this into a pickle object

In [ ]:
#First, creating the shortest paths for the base-case 
baseshortest = scr.create_shortest_paths(graph, od)
#Saving the basecase
fl.save_obj(baseshortest, 'baseshortest')

Next, update the shortest paths by removing an edge (disruption event) from the network: 

In [ ]:
for edge in graph.edges():
    name = ",".join(edge)
    shortest_paths = scr.update_shortest_paths(graph, edge, baseshortest)
    fl.save_obj(shortest_paths, name, 'shortest/')

Creating the n-1 models (disrupted models) for the edges in the network, assigning passengers and travel time

In [ ]:
for edge in graph.edgesz
    name = ",".join(edge)
    tempshortest = fl.load_obj(name, 'shortest/')
    tempgraphs, passengers = scr.n1_analysis(basegraphs, od, tempshortest, edge)
    fl.save_obj(tempgraphs, name, 'n1graphs/') #the networkx model containing passenger load estimates and capacity. 
    fl.save_obj(passengers, name, 'passengers/') #the amount of passengers traveling over the network, including disconnected passengers
    print('added for edge: {}'.format(name))

Similarly, we want to do the same for the basecase:  base estimate for passengers: 

In [ ]:
tempgraphs, passengers = scr.n1_analysis(basegraphs, od, baseshortest, None)
fl.save_obj(tempgraphs, 'basepassengers')
fl.save_obj(passengers, 'base', 'passengers/')

Since the calculation of the n-1 models was quite computationally heavy, the following code was saved to a python file and run on an AWS server, keeping in mind the files needed:

In [ ]:
import os.path
from os import path
import networkx as nx
import scripts2 as scr
import fileloader as fl
import dataframes as df
import time

graph = fl.get_network()
basegraphs = fl.load_obj('basegraphs')
od = fl.get_OD_LU()
for edge in graph.edges():
    starttime = time.time()
    edge = list(graph.edges())[i]
    name = ",".join(edge)
    if path.exists('Pickles/test/'+ name + '.pickle'): continue #if I already created this file, skip 
    tempshortest = fl.load_obj(name, 'shortest/')
    tempgraphs, passengers = scr.n1_analysis(basegraphs, od, tempshortest, edge)
    fl.save_obj(tempgraphs, name, 'awsgraphs/')
    fl.save_obj(passengers, name, 'passengers/')
    print('added for edge: {}, time taken {}'.format(name, starttime-time.time()))